In [ ]:
import seaborn as sns
import polars as pl

# Load the Flights dataset from Seaborn into a Polars dataframe
flights = sns.load_dataset('flights')
df_flights = pl.DataFrame(flights)
df_flights

year,month,passengers
i64,cat,i64
1949,"""Jan""",112
1949,"""Feb""",118
1949,"""Mar""",132
1949,"""Apr""",129
1949,"""May""",121
1949,"""Jun""",135
1949,"""Jul""",148
1949,"""Aug""",148
1949,"""Sep""",136


In [ ]:
# Manual mapping of month abbreviations to numbers
month_map = {
    "Jan": 1, "Feb": 2, "Mar": 3,
    "Apr": 4, "May": 5, "Jun": 6,
    "Jul": 7, "Aug": 8, "Sep": 9,
    "Oct": 10, "Nov": 11, "Dec": 12
}

# Apply the mapping to the month column to convert abbreviations to numbers
df_flights_with_month_num = df_flights.with_columns(
    pl.col("month").map_elements(lambda x: month_map[x]).alias("month_num")
)

# Create the date column by combining year, month_num, and a fixed day (assuming the first day of the month)
df_flights_with_date = df_flights_with_month_num.with_columns(
    pl.datetime("year", "month_num", pl.lit(1)).alias("date")
)

df_flights_with_date

<ipython-input-9-27f59dc34dae>:11: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("month").map_elements(lambda x: ...)
with this one instead:
  + pl.col("month").replace(month_map)

  pl.col("month").map_elements(lambda x: month_map[x]).alias("month_num")


year,month,passengers,month_num,date
i64,cat,i64,i64,datetime[μs]
1949,"""Jan""",112,1,1949-01-01 00:00:00
1949,"""Feb""",118,2,1949-02-01 00:00:00
1949,"""Mar""",132,3,1949-03-01 00:00:00
1949,"""Apr""",129,4,1949-04-01 00:00:00
1949,"""May""",121,5,1949-05-01 00:00:00
1949,"""Jun""",135,6,1949-06-01 00:00:00
1949,"""Jul""",148,7,1949-07-01 00:00:00
1949,"""Aug""",148,8,1949-08-01 00:00:00
1949,"""Sep""",136,9,1949-09-01 00:00:00


In [ ]:
# Calculate a rolling average of passengers
rolling_avg_passengers = df_flights_with_date.with_columns(
    pl.col("passengers").rolling_mean(window_size=3).alias("rolling_avg_passengers")
)

rolling_avg_passengers

year,month,passengers,month_num,date,rolling_avg_passengers
i64,cat,i64,i64,datetime[μs],f64
1949,"""Jan""",112,1,1949-01-01 00:00:00,null
1949,"""Feb""",118,2,1949-02-01 00:00:00,null
1949,"""Mar""",132,3,1949-03-01 00:00:00,120.666667
1949,"""Apr""",129,4,1949-04-01 00:00:00,126.333333
1949,"""May""",121,5,1949-05-01 00:00:00,127.333333
1949,"""Jun""",135,6,1949-06-01 00:00:00,128.333333
1949,"""Jul""",148,7,1949-07-01 00:00:00,134.666667
1949,"""Aug""",148,8,1949-08-01 00:00:00,143.666667
1949,"""Sep""",136,9,1949-09-01 00:00:00,144.0
